In [1]:
from scipy.misc import imread
from keras.utils import np_utils

#set up parameters will be used 

# input image dimensions
img_rows, img_cols = 227, 227

# number of channels, should try to get it.no matter what
img_channels = 1 

# normalization parameters
img_Norm = 255.0

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5105)
/usr/local/lib/python3.5/dist-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
import os
from PIL import Image
import numpy as np
from math import ceil
img_rows,img_cols=227,227
# generate random data of rate of split ########随后随机生成
NegetivePath = '/home/jupyter_1/Finalcombine/dataforAlextrain/nega'
truedataPath = '/home/jupyter_1/Finalcombine/dataforAlextrain/true'
splitRate = 0.6
Nelisting = os.listdir(NegetivePath)
nb_sampNe = np.size(Nelisting)
nb_NeTrain = ceil(splitRate*nb_sampNe)
nb_NeValidation = nb_sampNe-nb_NeTrain
# print(nb_NeTrain)
Polisting = os.listdir(truedataPath)
nb_sampPo = np.size(Polisting)
nb_PoTrain = ceil(splitRate*nb_sampPo)
nb_PoValidation = nb_sampPo - nb_PoTrain

#####################
# data path
datapath= '/home/jupyter_1/Finalcombine/Datasplit/'
NeTrainpath= '/home/jupyter_1/Finalcombine/Datasplit/train/nega'
NeValidaitionpath= '/home/jupyter_1/Finalcombine/Datasplit/validation/nega'
PoTrainpath= '/home/jupyter_1/Finalcombine/Datasplit/train/true'
PoValidaitionpath= '/home/jupyter_1/Finalcombine/Datasplit/validation/true'
print(nb_sampNe,nb_NeTrain,nb_NeValidation,nb_sampPo,nb_PoTrain,nb_PoValidation)
i =0
for file in Nelisting:
    i = i +1
    im = Image.open(NegetivePath + '/' + file)   
    img = im.resize((img_rows,img_cols))
    gray = img   
    if i <= nb_NeTrain:
        gray.save( NeTrainpath+'/' +  file, "JPEG")
    else :
        gray.save( NeValidaitionpath+'/' +  file, "JPEG")
        
i=0            
for file in Polisting:
    i = i +1
    im = Image.open(truedataPath + '/' + file)   
    img = im.resize((img_rows,img_cols))
    gray = img    
    if i <= nb_PoTrain:
        gray.save( PoTrainpath+'/' +  file, "JPEG")
    else:
        gray.save( PoValidaitionpath+'/' +  file, "JPEG")


42 26 16 128 77 51


In [3]:
# model ALexnet 
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Reshape, Permute, Activation,Input, merge,normalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D,AtrousConvolution2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam,SGD

model = Sequential()

model.add(Convolution2D(96, 11, 11,subsample=(4,4),activation='relu',input_shape = (1,227,227)))
model.add(ZeroPadding2D(padding=(2, 2)))
model.add(MaxPooling2D((3, 3), strides=(2,2)))
model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=1))


model.add (Convolution2D(256,5,5,subsample=(1,1),activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(MaxPooling2D((3, 3), strides=(2,2)))
model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=1))

model.add(Convolution2D(384,3,3,subsample=(1,1)))
model.add(ZeroPadding2D(padding=(1, 1), dim_ordering='th'))

model.add(Convolution2D(384,3,3,subsample=(1,1)))
model.add(ZeroPadding2D(padding=(1, 1), dim_ordering='th'))

model.add(AtrousConvolution2D(256,3,3,subsample=(1,1)))

model.add(MaxPooling2D((3, 3), strides=(2,2)))

model.add(Convolution2D(4096,5,5,activation="relu"))
model.add(Convolution2D(4096,1,1,activation="relu"))
# model.add(AtrousConvolution2D(2,1,1))
model.add( Flatten() )
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2,activation='sigmoid'))


sgd = SGD(lr=0.01, momentum=0.5, decay=0.1, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
model.summary()



____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 96, 55, 55)    11712       convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 96, 59, 59)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 96, 29, 29)    0           zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 96, 29, 29)    192         maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [35]:
# this is the augmentation configuration we will use for training
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rotation_range=5.,
             width_shift_range=0.1,
             height_shift_range=0.1,
            
             zoom_range=0.2)

# this is the augmentation configuration we will use for testing:
# only rescaling
train_data_dir = '/home/jupyter_1/Finalcombine/Datasplit/train'
validation_data_dir = '/home/jupyter_1/Finalcombine/Datasplit/validation'

test_datagen = ImageDataGenerator(rotation_range=5.,
             width_shift_range=0.1,
             height_shift_range=0.1,
             
             zoom_range=0.2)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        shuffle = True,
        batch_size=1,
        color_mode='grayscale',
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=1,
        color_mode='grayscale',
        shuffle = True,
        class_mode='categorical')


Found 103 images belonging to 2 classes.
Found 67 images belonging to 2 classes.


In [36]:
model.fit_generator(
        train_generator,
        samples_per_epoch =2000,
        
        nb_epoch=5,
        validation_data=validation_generator,
        nb_val_samples = 800,
        )

Epoch 1/5
2000/2000 [==============================] - 307s - loss: 0.3307 - acc: 0.8528 - val_loss: 0.3548 - val_acc: 0.8037
Epoch 2/5
2000/2000 [==============================] - 301s - loss: 0.2488 - acc: 0.8835 - val_loss: 0.3590 - val_acc: 0.8237
Epoch 3/5
2000/2000 [==============================] - 271s - loss: 0.2365 - acc: 0.8910 - val_loss: 0.3284 - val_acc: 0.8425
Epoch 4/5
2000/2000 [==============================] - 279s - loss: 0.2151 - acc: 0.8985 - val_loss: 0.3546 - val_acc: 0.8331
Epoch 5/5
2000/2000 [==============================] - 278s - loss: 0.2074 - acc: 0.9085 - val_loss: 0.3383 - val_acc: 0.8469


In [8]:
# from keras.models import model_from_yaml
# model_yaml = model.to_yaml()
# with open("model.yaml", "w") as yaml_file:
#     yaml_file.write(model_yaml)
# # serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")
# yaml_file = open('model.yaml', 'r')
# loaded_model_yaml = yaml_file.read()
# yaml_file.close()
# loaded_model = model_from_yaml(loaded_model_yaml)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")

Saved model to disk
Loaded model from disk


In [38]:
from keras.models import model_from_yaml
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model1.h5")
print("Saved model to disk")
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model1.h5")
print("Loaded model from disk")

Saved model to disk
Loaded model from disk


In [39]:
# final 
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( 
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False)
train_data_dir = '/home/jupyter_1/Finalcombine/data/'
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        shuffle = False,
        batch_size=1,
        color_mode='grayscale',
        class_mode='categorical')

Found 712 images belonging to 2 classes.


In [11]:
loaded_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy', 'f1score', 'precision', 'recall'])
score = loaded_model.evaluate_generator(train_generator,val_samples=600)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[3], score[3]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[4], score[4]*100))

acc: 88.75%
precision: 88.75%
recall: 88.83%


In [40]:
loaded_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy', 'f1score', 'precision', 'recall'])
score = loaded_model.evaluate_generator(train_generator,val_samples=600)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[3], score[3]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[4], score[4]*100))

acc: 85.92%
precision: 85.92%
recall: 86.00%


In [12]:
# final 
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( 
        shear_range=0.,
        zoom_range=0.,
        rotation_range=0.,
        width_shift_range=0.,
        height_shift_range=0.,
        horizontal_flip=False)
train_data_dir = '/home/jupyter_1/Finalcombine/data/'
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        shuffle = False,
        batch_size=1,
        color_mode='grayscale',
        class_mode='categorical')

Found 712 images belonging to 2 classes.


In [13]:
loaded_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy', 'f1score', 'precision', 'recall'])
score = loaded_model.evaluate_generator(train_generator,val_samples=600)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[3], score[3]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[4], score[4]*100))

acc: 88.08%
precision: 88.08%
recall: 88.17%


In [14]:
# final 
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( 
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=0.,
        width_shift_range=0.,
        height_shift_range=0.,
        horizontal_flip=False)
train_data_dir = '/home/jupyter_1/Finalcombine/data/'
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        shuffle = False,
        batch_size=1,
        color_mode='grayscale',
        class_mode='categorical')

Found 712 images belonging to 2 classes.


In [15]:
loaded_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy', 'f1score', 'precision', 'recall'])
score = loaded_model.evaluate_generator(train_generator,val_samples=600)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[3], score[3]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[4], score[4]*100))

acc: 87.83%
precision: 87.75%
recall: 87.83%


In [16]:
loaded_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy', 'f1score', 'precision', 'recall'])
score = loaded_model.evaluate_generator(train_generator,val_samples=6000)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[3], score[3]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[4], score[4]*100))

acc: 87.62%
precision: 87.61%
recall: 87.68%


In [17]:
# final 
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( 
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=2.,
        width_shift_range=0.,
        height_shift_range=0.,
        horizontal_flip=False)
train_data_dir = '/home/jupyter_1/Finalcombine/data/'
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        shuffle = False,
        batch_size=1,
        color_mode='grayscale',
        class_mode='categorical')

Found 712 images belonging to 2 classes.


In [18]:
loaded_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy', 'f1score', 'precision', 'recall'])
score = loaded_model.evaluate_generator(train_generator,val_samples=600)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[3], score[3]*100))
print("%s: %.2f%%" % (loaded_model.metrics_names[4], score[4]*100))

acc: 87.83%
precision: 87.83%
recall: 87.83%
